## Домашнее задание: «Оценка точности модели, переобучение, регуляризация»

Необходимые импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve

In [ ]:
# 1. Загрузка и преобразование данных
df = pd.read_csv('athletes.csv')

# a. Проверка пропущенных значений
print("Пропущенные значения:")
print(df.isnull().sum())

# Удаление строк с пропусками в целевой переменной или важных признаках
df = df.dropna(subset=['sex'])

# Заполнение числовых признаков средним значением
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Заполнение категориальных признаков модой
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col != 'sex':  # кроме целевой переменной
        df[col] = df[col].fillna(df[col].mode()[0])

# b. Кодирование категориальных переменных
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    if col != 'sex':  # целевую переменную закодируем отдельно
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

# Кодирование целевой переменной (пол)
sex_encoder = LabelEncoder()
df['sex_encoded'] = sex_encoder.fit_transform(df['sex'])

# Выбор признаков и целевой переменной
# Исключаем нерелевантные колонки
X = df.drop(['sex', 'sex_encoded', 'id', 'name'], axis=1, errors='ignore')
y = df['sex_encoded']

# 2. Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение логистической регрессии
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# Предсказания вероятностей
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# 3. ROC-кривая с помощью sklearn
fpr_sklearn, tpr_sklearn, thresholds_sklearn = roc_curve(y_test, y_pred_proba)

# 4. ROC-AUC метрика с помощью sklearn
roc_auc_sklearn = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC (sklearn): {roc_auc_sklearn:.4f}")

# 5. Подсчет TPR и FPR вручную
def calculate_tpr_fpr(y_true, y_scores):
    # Сортируем по убыванию вероятности
    sorted_indices = np.argsort(y_scores)[::-1]
    y_true_sorted = y_true.iloc[sorted_indices]
    y_scores_sorted = y_scores[sorted_indices]
    
    # Уникальные пороги
    thresholds = np.unique(y_scores_sorted)
    thresholds = np.append(thresholds, 1.1)  # добавляем крайнее значение
    
    tpr_list = []
    fpr_list = []
    
    for threshold in thresholds:
        # Предсказания на основе порога
        y_pred = (y_scores_sorted >= threshold).astype(int)
        
        # Расчет матрицы ошибок
        tp = np.sum((y_pred == 1) & (y_true_sorted == 1))
        fp = np.sum((y_pred == 1) & (y_true_sorted == 0))
        fn = np.sum((y_pred == 0) & (y_true_sorted == 1))
        tn = np.sum((y_pred == 0) & (y_true_sorted == 0))
        
        # Расчет TPR и FPR
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    
    return np.array(fpr_list), np.array(tpr_list), thresholds

# Расчет метрик вручную
fpr_manual, tpr_manual, thresholds_manual = calculate_tpr_fpr(y_test, y_pred_proba)

# 6. Объединенный график ROC-кривых
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(fpr_sklearn, tpr_sklearn, 'b-', label=f'ROC sklearn (AUC = {roc_auc_sklearn:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Случайный классификатор')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая (sklearn)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(fpr_manual, tpr_manual, 'r-', label='ROC ручной расчет', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Случайный классификатор')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая (ручной расчет)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Совмещенный график для сравнения
plt.figure(figsize=(8, 6))
plt.plot(fpr_sklearn, tpr_sklearn, 'b-', label=f'sklearn (AUC = {roc_auc_sklearn:.3f})', linewidth=2)
plt.plot(fpr_manual, tpr_manual, 'r--', label='Ручной расчет', linewidth=2, alpha=0.7)
plt.plot([0, 1], [0, 1], 'k--', label='Случайный классификатор')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Сравнение ROC-кривых')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 7. Precision-Recall кривая (вручную)
def calculate_precision_recall(y_true, y_scores):
    sorted_indices = np.argsort(y_scores)[::-1]
    y_true_sorted = y_true.iloc[sorted_indices]
    y_scores_sorted = y_scores[sorted_indices]
    
    thresholds = np.unique(y_scores_sorted)
    thresholds = np.append(thresholds, 1.1)
    
    precision_list = []
    recall_list = []
    
    for threshold in thresholds:
        y_pred = (y_scores_sorted >= threshold).astype(int)
        
        tp = np.sum((y_pred == 1) & (y_true_sorted == 1))
        fp = np.sum((y_pred == 1) & (y_true_sorted == 0))
        fn = np.sum((y_pred == 0) & (y_true_sorted == 1))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 1
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        precision_list.append(precision)
        recall_list.append(recall)
    
    return np.array(precision_list), np.array(recall_list), thresholds

precision_manual, recall_manual, _ = calculate_precision_recall(y_test, y_pred_proba)

# Precision-Recall кривая
plt.figure(figsize=(8, 6))
plt.plot(recall_manual, precision_manual, 'g-', linewidth=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall кривая (ручной расчет)')
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.show()

# 8. Расчет ROC-AUC вручную (методом трапеций)
def calculate_auc_manual(fpr, tpr):
    # Сортируем точки по FPR
    sorted_indices = np.argsort(fpr)
    fpr_sorted = fpr[sorted_indices]
    tpr_sorted = tpr[sorted_indices]
    
    # Расчет AUC методом трапеций
    auc = 0
    for i in range(1, len(fpr_sorted)):
        width = fpr_sorted[i] - fpr_sorted[i-1]
        height = (tpr_sorted[i] + tpr_sorted[i-1]) / 2
        auc += width * height
    
    return auc

auc_manual = calculate_auc_manual(fpr_manual, tpr_manual)
print(f"ROC-AUC (ручной расчет): {auc_manual:.4f}")

# 9. Выводы
print("\n" + "="*60)
print("ВЫВОДЫ")
print("="*60)

print("\na) Оценка качества модели:")
print("-" * 40)
print(f"1. ROC-AUC метрика: {roc_auc_sklearn:.4f}")
print("2. Интерпретация ROC-AUC:")
print("   - 0.9-1.0: Отличное качество")
print("   - 0.8-0.9: Хорошее качество")
print("   - 0.7-0.8: Удовлетворительное")
print("   - 0.6-0.7: Плохое")
print("   - 0.5-0.6: Не лучше случайного")

print("\n3. Анализ ROC-кривой:")
if roc_auc_sklearn > 0.8:
    print("   ✓ Кривая близка к левому верхнему углу - модель хорошая")
    print("   ✓ Модель значительно лучше случайного классификатора")
elif roc_auc_sklearn > 0.7:
    print("   ○ Кривая выше диагонали - модель имеет предсказательную способность")
    print("   ○ Есть потенциал для улучшения")
else:
    print("   ⚠ Модель близка к случайному угадыванию")

print("\n4. Precision-Recall анализ:")
# Расчет Average Precision
avg_precision = np.mean(precision_manual[recall_manual > 0])
print(f"   Средняя точность: {avg_precision:.3f}")

print("\nb) Может ли ROC-кривая проходить ниже диагонали?")
print("-" * 40)
print("Теоретически ROC-кривая может проходить ниже диагонали, если модель")
print("работает ХУЖЕ случайного классификатора. На практике это означает:")
print("1. Можно инвертировать предсказания (заменить 0 на 1 и наоборот)")
print("2. После инверсии кривая окажется выше диагонали")
print("3. AUC < 0.5 указывает на такое поведение")
print(f"4. В нашей модели AUC = {roc_auc_sklearn:.4f} > 0.5, поэтому кривая выше диагонали")